In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
pd.set_option('max_columns', 1000)
from tqdm import tqdm
from sklearn.neighbors import BallTree
import math
from scipy.spatial import Voronoi, voronoi_plot_2d
from datetime import datetime
import pytz
from IPython.display import HTML
import scipy.stats as stats
import matplotlib as mpl
from matplotlib import animation, rc, use
from matplotlib.patches import Rectangle, Arrow
import tensorflow as tf
from matplotlib.patches import Polygon
import matplotlib.patheffects as pe
import gc


def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    
    return df


def get_dx_dy(radian_angle, dist):
    dx = dist * math.cos(radian_angle)
    dy = dist * math.sin(radian_angle)
    return dx, dy


def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=50,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12*2, 6.33*2)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='slategrey', zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.3,
                                edgecolor='k',
                                facecolor='royalblue',
                                alpha=0.4,
                                zorder=1)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.3,
                                edgecolor='k',
                                facecolor='royalblue',
                                alpha=0.4,
                                zorder=1)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(0, 53.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
                 color='yellow')
    return fig, ax



class CreateNFLData:

    def __init__(self):
        pass

    def LoadData(self, Normal=True):
        if Normal == True:
            print("Loading Original Data")
            globbed_files = glob.glob("week*.csv") #creates a list of all csv files
            data = []
            for csv in tqdm(globbed_files):
                frame = pd.read_csv(csv, index_col=0)
                data.append(frame)

            WeekData = pd.concat(data).reset_index()
            WeekData
        
        else:
            print("Loading Modified Data")
            globbed_files = glob.glob("Revised Data/*.csv") #creates a list of all csv files
            data = []
            for csv in tqdm(globbed_files):
                frame = pd.read_csv(csv, index_col=0)
                data.append(frame)

            WeekData = pd.concat(data).reset_index()
            WeekData
        return WeekData



    def Standardize(self,W):
        print("Standardizing Data..")
        W['Dir_rad'] = np.mod(90 - W.dir, 360) * math.pi/180.0
        W['ToLeft'] = W.playDirection == "left"
        W['TeamOnOffense'] = "home"
        W.loc[W.possessionTeam != W.PlayerTeam, 'TeamOnOffense'] = "away"
        W['IsOnOffense'] = W.PlayerTeam == W.TeamOnOffense # Is player on offense?
        W['YardLine_std'] = 100 - W.yardlineNumber
        W.loc[W.yardlineSide.fillna('') == W.possessionTeam,  
                'YardLine_std'
                ] = W.loc[W.yardlineSide.fillna('') == W.possessionTeam,  
                'yardlineNumber']
        W['X_std'] = W.x
        W.loc[W.ToLeft, 'X_std'] = 120 - W.loc[W.ToLeft, 'x'] 
        W['Y_std'] = W.y
        W.loc[W.ToLeft, 'Y_std'] = 160/3 - W.loc[W.ToLeft, 'y'] 
        #W['Orientation_std'] = -90 + W.Orientation
        #W.loc[W.ToLeft, 'Orientation_std'] = np.mod(180 + W.loc[W.ToLeft, 'Orientation_std'], 360)
        W['Dir_std'] = W.Dir_rad
        W.loc[W.ToLeft, 'Dir_std'] = np.mod(np.pi + W.loc[W.ToLeft, 'Dir_rad'], 2*np.pi)
        W['dx'] = round(W['s']*np.cos(W['Dir_std']),2)
        W['dy'] = round(W['s']*np.sin(W['Dir_std']),2)
        W['X_std'] = round(W['X_std'],2)
        W['Y_std'] = round(W['Y_std'],2)
        #W['Orientation_rad'] = np.mod(W.o, 360) * math.pi/180.0
        W['Orientation_rad'] = np.mod(-W.o + 90, 360) * math.pi/180.0
        W['Orientation_std'] = W.Orientation_rad
        W.loc[W.ToLeft, 'Orientation_std'] = np.mod(np.pi + W.loc[W.ToLeft, 'Orientation_rad'], 2*np.pi)
        W['MPH'] = W['s'] / 0.488889
        return W

    
    def FrameData(self,WeekData1):
        NotNone = WeekData1.query('event != "None"')
        NotNone = NotNone.groupby(['gameId','playId','event'])['frameId'].max().reset_index()
        NotNone = NotNone.set_index(['gameId','playId','event'], drop= True).unstack('event').reset_index()
        NotNone.columns = [' '.join(col).strip() for col in NotNone.columns.values]
        NotNone.columns = NotNone.columns.str.replace('frameId' , '')
        NotNone.columns = NotNone.columns.str.replace(' ' , '')
        NotNone['Code'] = NotNone['gameId'].astype(str) + "-" + NotNone['playId'].astype(str)
        NotNone = NotNone.set_index('Code')
        NotNone = NotNone.loc[~NotNone.index.duplicated(keep='first')]

        for col in tqdm(NotNone.columns):
            NotNone['Contains_' + str(col)] = np.where(NotNone[col] > 0, True, False)

        Cols = ['ball_snap', 'man_in_motion', 'pass_arrived', 'pass_forward','pass_outcome_caught', 'play_action', 'run_pass_option', 'Contains_man_in_motion', 'Contains_pass_arrived', 'Contains_pass_forward', 'Contains_pass_outcome_caught', 'Contains_play_action','Contains_run_pass_option']
    #   WeekData1 = pd.merge(df, NotNone, how="left", left_on=['gameId','playId'], right_on=['gameId','playId'] )
        for col in Cols:
            WeekData1[col] = WeekData1.Code.map(NotNone[col])

        del NotNone
        gc.collect()

        WeekData1['After_snap'] = np.where(WeekData1['frameId'] > WeekData1['ball_snap'],1,0)
        WeekData1['After_Throw'] = np.where(WeekData1['frameId'] > WeekData1['pass_forward'],1,0)
        WeekData1['After_PassArrived'] = np.where(WeekData1['frameId'] > WeekData1['pass_arrived'],1,0)
        WeekData1['After_PlayAction'] = np.where(WeekData1['frameId'] > WeekData1['play_action'],1,0)
    #   WeekData1['After_run_pass_option'] = np.where(WeekData1['frameId'] > WeekData1['run_pass_option'],1,0)
        WeekData1['After_Catch'] = np.where(WeekData1['frameId'] > WeekData1['pass_outcome_caught'],1,0)

        
        LOS = WeekData1.query('displayName == "Football" & After_snap == 0')
        LOS = LOS.groupby(['gameId','playId'])['X_std','Y_std'].agg('median').reset_index()
        LOS.columns = ['gameId','playId','LOSX','LOSY']
        LOS['Code'] = LOS['gameId'].astype(str) + "-" + LOS['playId'].astype(str)
        LOS = LOS.set_index('Code')
        LOS = LOS.loc[~LOS.index.duplicated(keep='first')]
        WeekData1["LOSX"] = WeekData1.Code.map(LOS['LOSX'])
        WeekData1["LOSY"] = WeekData1.Code.map(LOS['LOSY'])
        WeekData1['Distfrom_LOSX'] = WeekData1['X_std'] - WeekData1['LOSX']
        WeekData1['Distfrom_LOSY'] = WeekData1['Y_std'] - WeekData1['LOSY']
        WeekData1['AbsDistfrom_LOSX'] = np.abs(WeekData1['X_std'] - WeekData1['LOSX'])
        WeekData1['AbsDistfrom_LOSY'] = np.abs(WeekData1['Y_std'] - WeekData1['LOSY'])
        del LOS
        gc.collect()
        return WeekData1
    
    def import_data(self,file,columns=False,cols=""):
        """create a dataframe and optimize its memory usage"""
        if columns == False:
            df = pd.read_csv(file, low_memory=False)
            df = reduce_mem_usage(df)
        else:
            df = pd.read_csv(file, low_memory=False, usecols=cols)
            df = reduce_mem_usage(df)
        return df
    
    






class AnimatePlay:
    def __init__(self, play_df,player_id=[], Tri = False, MPH = False,Text="",Show='jerseyNumber',method='all' ) -> None:
        self._MAX_FIELD_Y = 53.3
        self._MAX_FIELD_X = 120
        self._MAX_FIELD_PLAYERS = 22
        

        self.Tri = Tri
        self.MPH = MPH
        self.player_id = player_id
        self.Show = Show
        self.method = method
        self.Text = Text

        self._CPLT = sns.color_palette("husl", 2)
        self._frame_data = play_df
        self._times = sorted(play_df.time.unique())
        self._stream = self.data_stream()
        
        self._date_format = "%Y-%m-%dT%H:%M:%S.%fZ" 
        self._mean_interval_ms = np.mean([delta.microseconds/1000 for delta in np.diff(np.array([pytz.timezone('US/Eastern').localize(datetime.strptime(date_string, self._date_format)) for date_string in self._times]))])
        
        self._fig, self._ax_field = create_football_field()

        self._fig.set_figheight(10)
        self._fig.set_figwidth(15)
        
        self._fig.tight_layout()
        
        self._ax_field = plt.gca()
        
        self._ax_home = self._ax_field.twinx()
        self._ax_away = self._ax_field.twinx()
        self._ax_jersey = self._ax_field.twinx()

        self.ani = animation.FuncAnimation(self._fig, self.update, frames=len(self._times), interval = self._mean_interval_ms, 
                                          init_func=self.setup_plot, blit=False)
        
        plt.close()
       
    @staticmethod
    def set_axis_plots(ax, max_x, max_y) -> None:
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

        ax.set_xlim([0, max_x])
        ax.set_ylim([0, max_y])
        
    @staticmethod
    def convert_orientation(x):
        return (-x + 90)%360
    
    @staticmethod
    def polar_to_z(r, theta):
        return r * np.exp( 1j * theta)
    
    @staticmethod
    def deg_to_rad(deg):
        return deg*np.pi/180
        
    def data_stream(self):
        for time in self._times:
            yield self._frame_data[self._frame_data.time == time]
    
    def setup_plot(self): 
        self.set_axis_plots(self._ax_field, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        
        ball_snap_df = self._frame_data[(self._frame_data.event == 'ball_snap') & (self._frame_data.team == 'football')]
        self._ax_field.axvline(ball_snap_df.X_std.to_numpy()[0], color = 'yellow', linestyle = '--')
        
        self.set_axis_plots(self._ax_home, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        self.set_axis_plots(self._ax_away, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        self.set_axis_plots(self._ax_jersey, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        
        for idx in range(10,120,10):
            self._ax_field.axvline(idx, color = 'k', linestyle = '-', alpha = 0.05)
            
        self._scat_field = self._ax_field.scatter([], [], s = 200, color = 'red')
        self._scat_home = self._ax_home.scatter([], [], s = 900, color = self._CPLT[0], edgecolors = 'k')
        self._scat_away = self._ax_away.scatter([], [], s = 900, color = self._CPLT[1], edgecolors = 'k')
        
        self._scat_jersey_list = []
        self._scat_number_list = []
        self._scat_name_list = []
        self._scat_mph_list = []
        self._a_dir_list = []
        self._a_or_list = []
        self._a_tri_list = []
        for _ in range(self._MAX_FIELD_PLAYERS):
            self._scat_jersey_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black',fontweight='bold',fontsize='large',path_effects=[pe.withStroke(linewidth=3, foreground="white")]))
            self._scat_number_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'white',fontweight='bold',fontsize=14,path_effects=[pe.withStroke(linewidth=5, foreground="dodgerblue")]))
            self._scat_name_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black',fontweight='bold',fontsize='larger',path_effects=[pe.withStroke(linewidth=5, foreground="gold")]))
            self._scat_mph_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'lime',fontweight='bold',fontsize='larger'))

            self._a_dir_list.append(self._ax_field.add_patch(Arrow(0, 0, 0, 0, color = 'k')))
            self._a_or_list.append(self._ax_field.add_patch(Arrow(0, 0, 0, 0, color = 'k')))
            self._a_tri_list.append(self._ax_field.add_patch(Arrow(0, 0, 0, 0, color = 'k')))
            
        return (self._scat_field, self._scat_home, self._scat_away,*self._scat_mph_list, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)
        
    def update(self, anim_frame):
        pos_df = next(self._stream)
        
        for label in pos_df.team.unique():
            label_data = pos_df[pos_df.team == label]

            if label == 'football':
                self._scat_field.set_offsets(np.hstack([label_data.X_std, label_data.Y_std]))
            elif label == 'home':
                self._scat_home.set_offsets(np.vstack([label_data.X_std, label_data.Y_std]).T)
            elif label == 'away':
                self._scat_away.set_offsets(np.vstack([label_data.X_std, label_data.Y_std]).T)

        jersey_df = pos_df[pos_df.jerseyNumber.notnull()]
        
        for (index, row) in pos_df[pos_df.jerseyNumber.notnull()].reset_index().iterrows():
            self._scat_jersey_list[index].set_position((row.X_std, row.Y_std))
            self._scat_jersey_list[index].set_text(row.position)
            if self.method == 'single':
                try:
                    self._scat_number_list[index].set_text(np.where(np.isin(row.nflId,self.player_id) == True,str(self.Text) +" "+ str(round(row[self.Show],2)),""))
                    self._scat_number_list[index].set_position((row.X_std, row.Y_std+2.4))
                except:
                    self._scat_number_list[index].set_text(np.where(np.isin(row.nflId,self.player_id) == True,str(self.Text) +" "+ str(row[self.Show]),""))
                    self._scat_number_list[index].set_position((row.X_std, row.Y_std+2.4))
                    pass
            else:
                try:
                    self._scat_number_list[index].set_text(str(round(row[self.Show],2)))
                    self._scat_number_list[index].set_position((row.X_std, row.Y_std+2.4))
                except:
                    self._scat_number_list[index].set_text(row[self.Show])
                    self._scat_number_list[index].set_position((row.X_std, row.Y_std+2.4))
                    pass               

            self._scat_name_list[index].set_text(np.where(row.frameId <= 10,row.displayName.split()[-1],""))
            self._scat_name_list[index].set_position((row.X_std, row.Y_std-1.9))
            if self.MPH == True:
                self._scat_mph_list[index].set_text(np.where((row.s / 0.488889) > 17,str(round(float(row.s / 0.488889),2)) + " MPH",""))
                self._scat_mph_list[index].set_position((row.X_std, row.Y_std+1.9))
            else:
                pass

            player_vel = np.array([row.dx, row.dy])
            player_orient = np.array([np.real(self.polar_to_z(3, row.Orientation_std)), np.imag(self.polar_to_z(3, row.Orientation_std))])
            
            self._a_dir_list[index].remove()
            self._a_dir_list[index] = self._ax_field.add_patch(Arrow(row.X_std, row.Y_std, player_vel[0], player_vel[1], color = 'black'))
            
            self._a_or_list[index].remove()
            self._a_or_list[index] = self._ax_field.add_patch(Arrow(row.X_std, row.Y_std, player_orient[0], player_orient[1], color = 'blue', width = 2))

            if self.Tri == True:
                if (self.method == 'single') & (np.isin(row.nflId,self.player_id) == True):
                    self._a_tri_list[index].remove()
                    self._a_tri_list[index] = self._ax_field.add_patch(Polygon([[row.X_std, row.Y_std], [row.X_std_COpp,row.Y_std_COpp],[row.X_std_QB,row.Y_std_QB]], closed=True, fill=False, hatch='/',color='lime'))
                else:
      #              self._a_tri_list[index].remove()
      #              self._a_tri_list[index] = self._ax_field.add_patch(Polygon([[row.X_std, row.Y_std], [row.X_std_COpp,row.Y_std_COpp],[row.X_std_QB,row.Y_std_QB]], closed=True, fill=False, hatch='/',color='lime'))
                    pass
            else:
                pass
        
        return (self._scat_field, self._scat_home, self._scat_away, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import glob
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.patches as patches
pd.set_option('max_columns', 1000)

from sklearn.neighbors import BallTree

#from BDBUtils.Utilities import CreateNFLData
from IPython.core.display import HTML
import time
import math
from scipy.spatial import Voronoi, voronoi_plot_2d
from datetime import datetime
import pytz
from IPython.display import HTML
import scipy.stats as stats
import matplotlib as mpl
from matplotlib import animation, rc
from matplotlib.patches import Rectangle, Arrow
import tensorflow as tf
#from BDBUtils.Utilities import CreateNFLData

import glob
import os

np.set_printoptions(suppress=True)

import gc

def convert_orientation(x):
    return (x)%360

def deg_to_rad(deg):
        return deg*np.pi/180



Create = CreateNFLData()

start = time.process_time()

Weeks = range(1,18)

#globbed_files = glob.glob("../input/revised-data/*.csv") #creates a list of all csv files
data = []
for n in tqdm(Weeks):
    filename = '../input/revised-data/week' + str(n) + '.csv'
    frame = Create.import_data(filename,columns=True, cols=['week', 'gameId', 'playId', 'frameId', 'time', 'nflId', 'displayName', 'jerseyNumber', 'position', 'team', 'X_std', 'Y_std', 'Dir_std', 'dx', 'dy', 'Orientation_std', 's', 'MPH', 'a', 'dis', 'event','route', 'PlayerTeam','yardlineNumber', 'YardLine_std', 'OnOffense',  'closestOpp_Id', 'Opp_Dist_COpp','route_COpp', 'X_std_COpp', 'Y_std_COpp', 'Dir_std_COpp', 'dx_COpp', 'dy_COpp', 'Orientation_std_COpp', 'MPH_COpp', 's_COpp', 'a_COpp', 'dis_COpp', 'position_COpp', 'Pos_Rank_COpp','closestTeam_Id', 'Team_Dist_CTm', 'nflId_y_CTm', 'route_CTm', 'X_std_CTm', 'Y_std_CTm','QB_Dist_QB', 'X_std_QB', 'Y_std_QB', 'Orientation_std_QB', 'FootDist', 'Targeted'])
    frame['Code'] = frame['gameId'].astype(str) + "-" + frame['playId'].astype(str)
    plays = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv', usecols=['gameId', 'playId','down', 'yardsToGo','penaltyCodes', 'penaltyJerseyNumbers', 'passResult', 'offensePlayResult', 'playResult', 'epa', 'isDefensivePI','offenseFormation',	'personnelO',	'defendersInTheBox',	'personnelD',	'typeDropback','playType'])
    plays['Code'] = plays['gameId'].astype(str) + "-" + plays['playId'].astype(str)
    plays = plays.set_index('Code')
    plays = plays.loc[~plays.index.duplicated(keep='first')]
    Cols = ['playId','down', 'yardsToGo','penaltyCodes', 'penaltyJerseyNumbers', 'passResult', 'offensePlayResult', 'playResult', 'epa', 'isDefensivePI','offenseFormation',	'personnelO',	'defendersInTheBox',	'personnelD',	'typeDropback','playType']
    for col in Cols:
        frame[col] = frame.Code.map(plays[col])
    Obs = frame.select_dtypes(include=['object']).columns.to_list()
    frame[Obs] = frame[Obs].astype('category')
    data.append(frame)
    del plays
    gc.collect()
    del frame
    del Cols
    del Obs
    gc.collect()

print("finish")
WeekData = pd.concat(data).reset_index()
del data
gc.collect()


Finaldf1 = Create.FrameData(WeekData)
del WeekData
gc.collect()
Finaldf1.drop(['index','Code', 'ball_snap', 'man_in_motion', 'pass_arrived', 'pass_forward', 'pass_outcome_caught', 'play_action', 'run_pass_option'], axis=1, inplace=True)
Finaldf1.memory_usage().sum() / (1024**2)

gc.collect()
Finaldf1.memory_usage().sum() / (1024**2)

Finaldf1['QBslope'] = deg_to_rad(convert_orientation(np.rad2deg(np.arctan2(Finaldf1['Y_std_QB'] - Finaldf1['Y_std'], Finaldf1['X_std_QB'] - Finaldf1['X_std']))))
Finaldf1['WRslope'] = deg_to_rad(convert_orientation(np.rad2deg(np.arctan2(Finaldf1['Y_std_COpp'] - Finaldf1['Y_std'],Finaldf1['X_std_COpp'] - Finaldf1['X_std']))))

Finaldf1['QBslope1'] = convert_orientation(np.rad2deg(np.arctan2(Finaldf1['Y_std_QB'] - Finaldf1['Y_std'], Finaldf1['X_std_QB'] - Finaldf1['X_std'])))
Finaldf1['WRslope1'] = convert_orientation(np.rad2deg(np.arctan2(Finaldf1['Y_std_COpp'] - Finaldf1['Y_std'],Finaldf1['X_std_COpp'] - Finaldf1['X_std'])))
Finaldf1['Def_Or'] = convert_orientation(np.rad2deg(Finaldf1['Orientation_std']))
Finaldf1['Diff_QB'] = Finaldf1['QBslope1'] - Finaldf1['Def_Or']
Finaldf1['Diff_WR'] = Finaldf1['WRslope1'] - Finaldf1['Def_Or']

Finaldf1['Diff_QB'] = abs(np.where(Finaldf1['Diff_QB'] < -180,Finaldf1['Diff_QB'] + 360,Finaldf1['Diff_QB'] ))
Finaldf1['Diff_QB'] = abs(np.where(Finaldf1['Diff_QB'] > 180,Finaldf1['Diff_QB'] - 360,Finaldf1['Diff_QB'] ))

Finaldf1['Diff_WR'] = abs(np.where(Finaldf1['Diff_WR'] < -180,Finaldf1['Diff_WR'] + 360,Finaldf1['Diff_WR'] ))
Finaldf1['Diff_WR'] = abs(np.where(Finaldf1['Diff_WR'] > 180,Finaldf1['Diff_WR'] - 360,Finaldf1['Diff_WR'] ))

Finaldf1['Player_POV'] = np.where(Finaldf1['Diff_QB'] < Finaldf1['Diff_WR'],"QB",Finaldf1['position_COpp'])
Finaldf1['Looking_AtQB'] = np.where(Finaldf1['Diff_QB'] < Finaldf1['Diff_WR'],1,0)

Finaldf1['diffDir'] = np.absolute(Finaldf1['Dir_std'] - Finaldf1['Dir_std_COpp'])

Finaldf1['disRatio'] = Finaldf1['Opp_Dist_COpp'] / np.sqrt((Finaldf1['X_std_COpp'] - Finaldf1['X_std_CTm'])**2 + (Finaldf1['Y_std_COpp'] - Finaldf1['Y_std_CTm'])**2)

Finaldf1['Event2'] = np.where(Finaldf1['After_snap'] == 0,"Before Snap","After Snap - Before Throw")
Finaldf1['Event2'] = np.where((Finaldf1['After_Throw'] == 1 & (Finaldf1['After_PassArrived'] == 0)),"Ball in the Air", Finaldf1['Event2'])

Finaldf1['EventCount'] = Finaldf1.groupby(['gameId','playId','Event2'])['Event2'].transform('count') / Finaldf1.groupby(['gameId','playId'])['nflId'].transform('nunique')
Finaldf1['EventOrder'] = Finaldf1.groupby(['gameId','playId','Event2'])['frameId'].rank(ascending=True, method='dense').astype(int)
Finaldf1['EventPct'] = Finaldf1['EventOrder'] / Finaldf1['EventCount']

Finaldf1['Partition'] = np.where(Finaldf1['EventPct'] > (1/2), "2nd Phase", "1st Phase")

Finaldf1['Group'] = Finaldf1['Event2'] + "-" + Finaldf1['Partition']

Finaldf1['Player_POV'] = Finaldf1['Player_POV'].astype('category')
Finaldf1['Event2'] = Finaldf1['Event2'].astype('category')
Finaldf1['Partition'] = Finaldf1['Partition'].astype('category')
Finaldf1['Group'] = Finaldf1['Group'].astype('category')



In [ ]:
!pip install ipywidgets
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install chart_studio

In [ ]:
import plotly.figure_factory as ff
Routes = ['GO','HITCH','OUT','CROSS','IN','POST','SLANT','CORNER']

OffBall = Finaldf1.loc[(Finaldf1['Group'] == "After Snap - Before Throw-1st Phase") & (Finaldf1['position'] == "WR") & (Finaldf1['position_COpp'] == "CB") & (Finaldf1['EventOrder'] <= 15) ]


OffBall['MaxAccel'] = OffBall.groupby(['gameId','playId','nflId','route'])['a'].transform('max')
OffBall['MaxAccelFrame'] = np.where(OffBall['a'] == OffBall['MaxAccel'],OffBall['EventOrder'],0)
OffBall['MaxAccelFrame'] = OffBall.groupby(['gameId','playId','nflId','route'])['MaxAccelFrame'].transform('max')

OffBall['Maxdx'] = OffBall.groupby(['gameId','playId','nflId','route'])['dx'].transform('max')
OffBall['MaxdxFrame'] = np.where(OffBall['dx'] == OffBall['Maxdx'],OffBall['EventOrder'],0)
OffBall['MaxdxFrame'] = OffBall.groupby(['gameId','playId','nflId','route'])['MaxdxFrame'].transform('max')

OffBall['MaxSeparation'] = OffBall.groupby(['gameId','playId','nflId','route'])['Opp_Dist_COpp'].transform('max')
OffBall['MaxSeparationFrame'] = np.where(OffBall['Opp_Dist_COpp'] == OffBall['MaxSeparation'],OffBall['EventOrder'],0)
OffBall['MaxSeparationFrame'] = OffBall.groupby(['gameId','playId','nflId','route'])['MaxSeparationFrame'].transform('max')

OffBall['AccelDiff'] = OffBall['a'] - OffBall['a_COpp']
OffBall['SpeedDiff'] = OffBall['s'] - OffBall['s_COpp']
OffBall['dxDiff'] = OffBall['dx'] - OffBall['dx_COpp']
OffBall['dyDiff'] = OffBall['dy'] - OffBall['dy_COpp']



OffBall['MaxAccelDiff'] = OffBall.groupby(['gameId','playId','nflId','route'])['AccelDiff'].transform('max')
OffBall['MaxAccelDiffFrame'] = np.where(OffBall['AccelDiff'] == OffBall['MaxAccelDiff'],OffBall['EventOrder'],0)
OffBall['MaxAccelDiffFrame'] = OffBall.groupby(['gameId','playId','nflId','route'])['MaxAccelDiffFrame'].transform('max')

OffBall['MaxSpeedDiff'] = OffBall.groupby(['gameId','playId','nflId','route'])['SpeedDiff'].transform('max')
OffBall['MaxSpeedDiffFrame'] = np.where(OffBall['SpeedDiff'] == OffBall['MaxSpeedDiff'],OffBall['EventOrder'],0)
OffBall['MaxSpeedDiffFrame'] = OffBall.groupby(['gameId','playId','nflId','route'])['MaxSpeedDiffFrame'].transform('max')

OffBall['MaxdxDiff'] = OffBall.groupby(['gameId','playId','nflId','route'])['dxDiff'].transform('max')
OffBall['MaxdxDiffFrame'] = np.where(OffBall['dxDiff'] == OffBall['MaxdxDiff'],OffBall['EventOrder'],0)
OffBall['MaxdxDiffFrame'] = OffBall.groupby(['gameId','playId','nflId','route'])['MaxdxDiffFrame'].transform('max')

OffBall['MaxdyDiff'] = OffBall.groupby(['gameId','playId','nflId','route'])['dyDiff'].transform('max')
OffBall['MaxdyDiffFrame'] = np.where(OffBall['dyDiff'] == OffBall['MaxdyDiff'],OffBall['EventOrder'],0)
OffBall['MaxdyDiffFrame'] = OffBall.groupby(['gameId','playId','nflId','route'])['MaxdyDiffFrame'].transform('max')

OffBall['CatchSuccess'] = np.where(OffBall['passResult'] == "C",0,1)
OffBall['YardsSuccess'] = np.where(OffBall['offensePlayResult'] > 0,0,1)

# Press Coverage

Press coverage is a technique in which the cornerback lines up close to the wide receiver at the line of scrimmage when the ball is snapped, and tries to engage in contact to disrupt the route. 

Evaluating a cornerback's press coverage ability could take hours of film study and can be subjective at times, today we want to figure out how to quantify the cornerback's ability to jam / press the receiver at the line of scrimmage during the first seconds of the play.

---------------------------------------------------------------------------------------------------------------------------------------------------------------------

![](https://images.actionnetwork.com/blog/2018/11/brad.gif)

I will use the same graphic used in my piece on [shadow corners](https://www.kaggle.com/jdruzzi/shadow-cornerback-coverage-analysis). This is a textbook example of a good press coverage snap, the cornerback successfully slows down the timing of the receiver running the route by displaying his physicality at the line of scrimmage.

-------------------------------------------------------------------------------------------------------------------------

# Variables Used:

We will use the receivers velocity, acceleration and separation for each press coverage play and compare it to what they generally produce for each metric on the specific route run. Please note, that this is technically data leakage because we will use the receivers average for the season, which will already include the evaluated play metrics baked into the average. However, since we have an entire seasons worth of data, there are enough data points to get a solid idea of what the receiver generally produces on each press coverage snap. In a perfect world, with large amounts of historical data, we would use a lag rolling average or lag weighted moving average.



### **MVAE** - Max Velocity (dx) Above Expected
 - The max velocity relative to the x axis of the wide receiver, compared to the max velocity the receiver usually generates on that particular route during press coverage.


### **MAAE** - Max Acceleration Above Expected
- The max acceleration of the wide receiver, compared to the max acceleration the receiver usually generates on that particular route during press coverage

 
### **MVFAE** - Max Velocity Frame (dx) Above Expected 
- The max velocity relative to the x axis frame of the wide receiver, compared to the max velocity frame the receiver usually generates on that particular route during press coverage

 
### **MAFAE** - Max Acceleration Frame Above Expected
- The max acceleration frame of the wide receiver, compared to the max acceleration frame the receiver usually generates on that particular route during press coverage

 
### **MSFAE** - Max Separation Frame Above Expected 
- The max separation frame of the wide receiver, compared to the max separation frame the receiver usually generates on that particular route during press coverage


# Target Variable:

We will use an inverse completion probability or in our case, "Defender Success" to measure the effectiveness of the press coverage. We expect that a successful press coverage will result in an incomplete pass or interception.


### **Defender Success** 
- Binary variable with 0 = Completion Allowed, 1 = Incomplete pass or Interception

--------------------------------------------------------------------------------------------

# Criteria:

- We only analyze the first 1.5 seconds of the play, this is to focus solely on hand-to-hand combat at the line of scrimmage between the cornerback and the wide receiver

- Defender is within 3 yards of the receiver when the ball is snapped, indicating press coverage

- Defender is the closest defensive player relative to the wide receiver for the complete duration of the play, indicating man-to-man coverage



In [ ]:
MAAE = OffBall.groupby(['week','gameId','playId','nflId','displayName','PlayerTeam','closestOpp_Id','route','Targeted','offensePlayResult','passResult','YardLine_std','down']).agg({'Opp_Dist_COpp':[('start', 'first'),('last', 'last'), ('mean', 'mean'), ('count', 'count'),('max','max')],

                                                                                            'MPH':[('max', 'max'),('var', 'var'),('mean', 'mean'),],
                                                                                            'a':[('max', 'max'),('var', 'var'),('mean', 'mean')],
                                                                                            'dx':[('mean', 'mean'),('var','var'),('max','max')],
                                                                                            'dy':[('mean', 'mean'),('var','var'),('max','max')],
                                                                                            'dx_COpp':[('first', 'first'),('mean', 'mean'),('max','max')],
                                                                                            'dy_COpp':[('first', 'first'),('mean', 'mean'),('max','max')],
                                                                                            'a_COpp':[('mean', 'mean'),('max','max')],
                                                                                            'MaxAccelFrame':[('max', 'max')],
                                                                                            'epa':[('mean', 'mean')],
                                                                                            'MaxdxFrame':[('max', 'max')],
                                                                                            'MaxSeparationFrame':[('max', 'max')],
                                                                                            'MaxAccelDiffFrame':[('max', 'max')],
                                                                                            'MaxSpeedDiffFrame':[('max', 'max')],
                                                                                            'MaxdxDiffFrame':[('max', 'max')],
                                                                                            'MaxdyDiffFrame':[('max', 'max')],
                                                                                            'X_std':[('start', 'first'),('last', 'last')]}).reset_index(drop=False)
MAAE.columns = MAAE.columns.map('_'.join)
MAAE = MAAE.loc[MAAE['Opp_Dist_COpp_start'] < 3]
MAAE['count'] = MAAE.groupby(['nflId_','route_'])['displayName_'].transform('count')


MAAE['dx_dff'] = MAAE['dx_mean'] - MAAE['dx_COpp_mean']
MAAE['dy_dff'] = MAAE['dy_mean'] - MAAE['dx_COpp_mean']
MAAE['a_dff'] = MAAE['a_mean'] - MAAE['a_COpp_mean']


MAAE['MaxAccelbyFrame'] = MAAE['a_max'] / MAAE['MaxAccelFrame_max']
MAAE['MaxdxbyFrame'] = MAAE['dx_max'] / MAAE['MaxdxFrame_max']
MAAE['MaxSeparationbyFrame'] = MAAE['Opp_Dist_COpp_max'] / MAAE['MaxSeparationFrame_max']



MAAE['MovingForward'] = MAAE['X_std_last'] - MAAE['X_std_start']

MAAE['Total'] = MAAE.groupby(['gameId_','playId_','displayName_'])['Opp_Dist_COpp_count'].transform('sum')
MAAE['FramePct'] = MAAE['Opp_Dist_COpp_count'] / MAAE['Total']

MAAE = MAAE.query('MPH_max > 5 & Total >= 10 & Total <= 15 & FramePct == 1 & Opp_Dist_COpp_start <= 3 & Targeted_ == 1 & MovingForward > -1 & dx_COpp_first < .2 & Opp_Dist_COpp_last < 6').reset_index(drop=True)

Cols = ['Opp_Dist_COpp_start', 'Opp_Dist_COpp_last', 'Opp_Dist_COpp_mean',  'Opp_Dist_COpp_max', 'MPH_max', 'MPH_var', 'MPH_mean', 'a_max', 'a_var', 'dx_mean', 'dx_var', 'dx_max','dy_mean', 'dy_var', 'dy_max', 'dx_COpp_first','dx_COpp_mean', 'dx_COpp_max', 'MaxAccelFrame_max', 'MaxdxFrame_max', 'MaxSeparationFrame_max','dx_dff','dy_dff','a_dff','MaxAccelbyFrame','MaxdxbyFrame','MaxSeparationbyFrame','MaxAccelDiffFrame_max','MaxSpeedDiffFrame_max','MaxdxDiffFrame_max','MaxdyDiffFrame_max']

for col in Cols:
    MAAE[col + "_AE"] = MAAE[col] - MAAE.groupby(['nflId_','route_'])[col].transform('mean')



MAAE['PlaySuccess'] = np.where(MAAE['offensePlayResult_'] > 0, 0,1)
MAAE['DefenderSuccess'] = np.where(MAAE['passResult_'] == "C",0,1)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder,OneHotEncoder



Cols = ['dx_max_AE','a_max_AE', 'MaxAccelFrame_max_AE',  'MaxdxFrame_max_AE','MaxSeparationFrame_max_AE']

X = MAAE[Cols]

y = MAAE['DefenderSuccess']


X[Cols] = SimpleImputer().fit_transform(X[Cols])
X[Cols] = StandardScaler().fit_transform(X[Cols])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42, stratify=y)

--------------------------------------------------------

# Model

Instead of predicting catch probability, where completion = 1, and incomplete/interception = 0

We will predict Defender Success, where completion = 0, and incomplete/interception = 1

A simple model was generated using logistic regression. Even only analyzing the first 15 frames of the play, with no knowledge of the pass outcome, we were able to predict with roughly 58% accuracy whether it was a catch or not.

We can use the predicted probability of defender success, normalize it, and then multiply by 100 to create a scoring function.

**Press Score** = MinMaxScalar(P(Defender Success)) x 100

1-100 with 100 being the most ideal press coverage snap. A score above 50 would indicate a successful press coverage snap.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,auc,roc_auc_score,confusion_matrix,accuracy_score,f1_score
from sklearn.linear_model import RidgeClassifier

model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=65)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_clf_acc = grid_search.fit(X, y)

print("Best: %f using %s" % (grid_clf_acc.best_score_, grid_clf_acc.best_params_))

#Predict values based on new parameters
y_pred_acc = grid_clf_acc.predict(X_test)
y_pred_acctrain = grid_clf_acc.predict(X_train)
# New Model Evaluation metrics 
print('Train Accuracy Score : ' + str(accuracy_score(y_train,y_pred_acctrain)))
print('-----------------------------------------------')
print('Test Accuracy Score : ' + str(accuracy_score(y_test,y_pred_acc)))


MAAE['BasicSuccess_Prob'] = grid_clf_acc.predict_proba(X)[:,1]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

PressScore = OffBall.groupby(['week','gameId','playId','nflId','displayName','PlayerTeam','closestOpp_Id','route','Targeted','offensePlayResult','passResult','YardLine_std','down']).agg({'Opp_Dist_COpp':[('start', 'first'),('last', 'last'), ('mean', 'mean'), ('count', 'count'),('max','max')],
                                                                                            'MPH':[('max', 'max'),('var', 'var'),('mean', 'mean'),],
                                                                                            'a':[('max', 'max'),('var', 'var'),('mean', 'mean')],
                                                                                            'dx':[('mean', 'mean'),('var','var'),('max','max')],
                                                                                            'dy':[('mean', 'mean'),('var','var'),('max','max')],
                                                                                            'dx_COpp':[('first', 'first'),('mean', 'mean'),('max','max')],
                                                                                            'dy_COpp':[('first', 'first'),('mean', 'mean'),('max','max')],
                                                                                            'a_COpp':[('mean', 'mean'),('max','max')],
                                                                                            'MaxAccelFrame':[('max', 'max')],
                                                                                            'epa':[('mean', 'mean')],
                                                                                            'MaxdxFrame':[('max', 'max')],
                                                                                            'MaxSeparationFrame':[('max', 'max')],
                                                                                            'MaxAccelDiffFrame':[('max', 'max')],
                                                                                            'MaxSpeedDiffFrame':[('max', 'max')],
                                                                                            'MaxdxDiffFrame':[('max', 'max')],
                                                                                            'MaxdyDiffFrame':[('max', 'max')],
                                                                                            'X_std':[('start', 'first'),('last', 'last')]}).reset_index(drop=False)
PressScore.columns = PressScore.columns.map('_'.join)
PressScore = PressScore.loc[PressScore['Opp_Dist_COpp_start'] < 3]
PressScore['count'] = PressScore.groupby(['nflId_','route_'])['displayName_'].transform('count')


PressScore['dx_dff'] = PressScore['dx_mean'] - PressScore['dx_COpp_mean']
PressScore['dy_dff'] = PressScore['dy_mean'] - PressScore['dx_COpp_mean']
PressScore['a_dff'] = PressScore['a_mean'] - PressScore['a_COpp_mean']


PressScore['MaxAccelbyFrame'] = PressScore['a_max'] / PressScore['MaxAccelFrame_max']
PressScore['MaxdxbyFrame'] = PressScore['dx_max'] / PressScore['MaxdxFrame_max']
PressScore['MaxSeparationbyFrame'] = PressScore['Opp_Dist_COpp_max'] / PressScore['MaxSeparationFrame_max']

#PressScore['Press_Success'] = np.where((PressScore['PressScore'] < 0) & (PressScore['DistCOpp_AE'] < 0),1,0)



PressScore['Total'] = PressScore.groupby(['gameId_','playId_','displayName_'])['Opp_Dist_COpp_count'].transform('sum')
PressScore['FramePct'] = PressScore['Opp_Dist_COpp_count'] / PressScore['Total']

PressScore['PlaySuccess'] = np.where(PressScore['offensePlayResult_'] > 0, 0,1)
PressScore['DefenderSuccess'] = np.where(PressScore['passResult_'] == "C",0,1)


PressScore['MovingForward'] = PressScore['X_std_last'] - PressScore['X_std_start']


PressScore1 = PressScore.query('MPH_max > 5 & Total >= 10 & Total <= 15 & FramePct == 1 & Opp_Dist_COpp_start <= 3 & MovingForward > -1 & dx_COpp_first < .2 & Opp_Dist_COpp_last < 6 & Targeted_ == 1').reset_index(drop=True)


Cols = ['Opp_Dist_COpp_start', 'Opp_Dist_COpp_last', 'Opp_Dist_COpp_mean',  'Opp_Dist_COpp_max', 'MPH_max', 'MPH_var', 'MPH_mean', 'a_max', 'a_var', 'dx_mean', 'dx_var', 'dx_max','dy_mean', 'dy_var', 'dy_max', 'dx_COpp_first', 'dx_COpp_mean', 'dx_COpp_max', 'MaxAccelFrame_max', 'MaxdxFrame_max', 'MaxSeparationFrame_max','dx_dff','dy_dff','a_dff','MaxAccelbyFrame','MaxdxbyFrame','MaxSeparationbyFrame','MaxAccelDiffFrame_max','MaxSpeedDiffFrame_max','MaxdxDiffFrame_max','MaxdyDiffFrame_max']

for col in Cols:
    PressScore1[col + "_AE"] = PressScore1[col] - PressScore1.groupby(['nflId_','route_'])[col].transform('mean')






Cols =['dx_max_AE', 'a_max_AE','MaxAccelFrame_max_AE',  'MaxdxFrame_max_AE','MaxSeparationFrame_max_AE']


X = PressScore1[Cols]
#X = pd.get_dummies(X)


#Cols = ['MaxdxFrame_max', 'MaxSeparationFrame_max', 'MaxAccelFrame_max_AE',  'MaxdxFrame_max_AE']

y = PressScore1['DefenderSuccess']


X[Cols] = SimpleImputer().fit_transform(X[Cols])
X[Cols] = StandardScaler().fit_transform(X[Cols])



PressScore1['PressScore'] = grid_clf_acc.predict_proba(X)[:,1]
PressScore1['Press_Success'] = grid_clf_acc.predict(X)
PressScore1[['PressScore']] = MinMaxScaler().fit_transform(PressScore1[['PressScore']])
PressScore1['PressScore'] = PressScore1['PressScore']*100

In [ ]:
import chart_studio.plotly as py
import plotly.figure_factory as ff
import math

pd.options.display.float_format = '{:.4f}'.format

Press = PressScore1.filter(['gameId_','playId_','nflId_','dx_max_AE','dx_var_AE','dy_var_AE', 'a_max_AE','Opp_Dist_COpp_last_AE', 'MaxAccelFrame_max_AE',  'MaxdxFrame_max_AE','MaxSeparationFrame_max_AE','PressScore','Press_Success'], axis=1)
Press['Code'] = Press['gameId_'].astype(str) + "-" + Press['playId_'].astype(str) + "-" + Press['nflId_'].astype(str)
Press = Press.set_index('Code')
Press = Press.loc[~Press.index.duplicated(keep='first')]



def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]




OffBall['InitialDefDis'] = OffBall.groupby(['playId','nflId','closestOpp_Id'])['Opp_Dist_COpp'].transform('first')
OffBall['Press'] = np.where(OffBall['InitialDefDis'] < 3, 1,0)
#df = OffBall.query('displayName == "Mike Evans" & gameId == 2018110402 & playId ==558	').head(10).filter(['playId','displayName','route','EventOrder','a'],axis=1).rename(columns={'EventOrder':'Frames'})
#df = OffBall.query('displayName == "Michael Thomas" & gameId == 2018102101 & playId == 2235	').head(10).filter(['playId','displayName','route','EventOrder','a'],axis=1).rename(columns={'EventOrder':'Frames'})
df = OffBall.query('displayName == "Michael Thomas" & gameId == 2018110409 & playId == 3891	| displayName == "Mike Evans" & gameId == 2018110402 & playId == 558 | displayName == "Tyreek Hill" & gameId == 2018100705 & playId == 395 | displayName == "Pierre Garcon" & gameId == 2018101500 & playId == 1249| displayName == "Michael Crabtree" & gameId == 2018102101 & playId == 3444 | displayName == "Devin Funchess" & gameId == 2018121700 & playId == 2048 | displayName == "Davante Adams" & gameId == 2018102810 & playId == 412| displayName == "Mike Evans" & gameId == 2018090906 & playId == 3051 |  displayName == "Tyreek Hill" & gameId == 2018091605 & playId == 2430 ').filter(['gameId','playId','nflId','displayName','route','EventOrder','a','dx','Opp_Dist_COpp'],axis=1)#.rename(columns={'EventOrder':'Frames'})

#df = OffBall.query('displayName == "Mike Evans" & gameId == 2018110402 & playId == 558').filter(['playId','displayName','route','EventOrder','a','dx'],axis=1)#.rename(columns={'EventOrder':'Frames'})

df.columns = ['gameId','playId','nflId','Name','route','Frames','Accel','Velocity','Separation']

df["Accel"] = df['Accel'].map(lambda x: np.round(x,3))
df["Velocity"] = df['Velocity'].map(lambda x: np.round(x,3))
df["Separation"] = df['Separation'].map(lambda x: np.round(x,3))

df['AccelMax'] = OffBall.groupby(['displayName','route','Press'])['MaxAccel'].transform('mean')
df["AccelMax"] = df['AccelMax'].map(lambda x: np.round(x,3))

df['dxMax'] = OffBall.groupby(['displayName','route','Press'])['Maxdx'].transform('mean')
df["dxMax"] = df['dxMax'].map(lambda x: np.round(x,3))

df['MaxSeparation'] = OffBall.groupby(['displayName','route','Press'])['MaxSeparation'].transform('mean')
df["MaxSeparation"] = df['MaxSeparation'].map(lambda x: np.round(x,3))

df['MaxSeparationFrameAvg'] = OffBall.groupby(['displayName','route','Press'])['MaxSeparationFrame'].transform('mean')
df["MaxSeparationFrameAvg"] = df['MaxSeparationFrameAvg'].map(lambda x: np.round(x,3))

df['MaxdxFrameAvg'] = OffBall.groupby(['displayName','route','Press'])['MaxdxFrame'].transform('mean')
df["MaxdxFrameAvg"] = df['MaxdxFrameAvg'].map(lambda x: np.round(x,3))


df['MaxAccelFrameAvg'] = OffBall.groupby(['displayName','route','Press'])['MaxAccelFrame'].transform('mean')
df["MaxAccelFrameAvg"] = df['MaxAccelFrameAvg'].map(lambda x: np.round(x,3))

df['Code'] = df['gameId'].astype(str) + "-" + df['playId'].astype(str) + "-" + df['nflId'].astype(str)

df["Press Score"] = df.Code.map(np.round(Press["PressScore"],3))

del Press
gc.collect()


df['Matchup'] = np.where((df['Name'] == "Michael Thomas") & (df['gameId'] == 2018110409) & (df['playId'] == 3891), "Michael Thomas vs. Marcus Peters", 0)
df['Matchup'] = np.where((df['Name'] == "Mike Evans") & (df['gameId'] == 2018110402) & (df['playId'] == 558), "Mike Evans vs. James Bradberry", df['Matchup'])
df['Matchup'] = np.where((df['Name'] == "Tyreek Hill") & (df['gameId'] == 2018100705) & (df['playId'] == 395), "Tyreek Hill vs. Jalen Ramsey", df['Matchup'])
df['Matchup'] = np.where((df['Name'] == "Pierre Garcon") & (df['gameId'] == 2018101500) & (df['playId'] == 1249), "Pierre Garcon vs. Kevin King", df['Matchup'])
df['Matchup'] = np.where((df['Name'] == "Michael Crabtree") & (df['gameId'] == 2018102101) & (df['playId'] == 3444), "Michael Crabtree vs. Ken Crawley", df['Matchup'])
df['Matchup'] = np.where((df['Name'] == "Devin Funchess") & (df['gameId'] == 2018121700) & (df['playId'] == 2048), "Devin Funchess vs. Eli Apple", df['Matchup'])
df['Matchup'] = np.where((df['Name'] == "Davante Adams") & (df['gameId'] == 2018102810) & (df['playId'] == 412), "Davante Adams vs. Marcus Peters", df['Matchup'])
df['Matchup'] = np.where((df['Name'] == "Mike Evans") & (df['gameId'] == 2018090906) & (df['playId'] == 3051), "Mike Evans vs. Marshon Lattimore", df['Matchup'])
df['Matchup'] = np.where((df['Name'] == "Tyreek Hill") & (df['gameId'] == 2018091605) & (df['playId'] == 2430), "Tyreek Hill vs. Artie Burns", df['Matchup'])


df['link'] = np.where(df["Matchup"] == "Tyreek Hill vs. Jalen Ramsey", "https://media1.giphy.com/media/703yS4wdiLvJpzHion/giphy.gif", 0)
df['link'] = np.where(df["Matchup"] == "Michael Thomas vs. Marcus Peters", "https://media4.giphy.com/media/cXgBT9FPpHtGSxC4gY/giphy.gif", df['link'])
df['link'] = np.where(df["Matchup"] == "Mike Evans vs. James Bradberry", "https://images.actionnetwork.com/blog/2018/11/brad.gif", df['link'])
df['link'] = np.where(df["Matchup"] == "Pierre Garcon vs. Kevin King", "https://media3.giphy.com/media/Xdrtq0QgTiVGRBV1LH/giphy.gif", df['link'])
df['link'] = np.where(df["Matchup"] == "Michael Crabtree vs. Ken Crawley", "https://media0.giphy.com/media/CqO673JlHLKdnnEoRp/giphy.gif", df['link'])
df['link'] = np.where(df["Matchup"] == "Devin Funchess vs. Eli Apple", "https://media1.giphy.com/media/n8KgzfQQCHhFxbJUO5/giphy.gif", df['link'])

df['link'] = np.where(df["Matchup"] == "Davante Adams vs. Marcus Peters", "https://media2.giphy.com/media/eiWqmciwHLKiBdLmXT/giphy.gif", df['link'])
df['link'] = np.where(df["Matchup"] == "Mike Evans vs. Marshon Lattimore", "https://media2.giphy.com/media/Nwuo3dpeddNuxZh7e2/giphy.gif", df['link'])
df['link'] = np.where(df["Matchup"] == "Tyreek Hill vs. Artie Burns", "https://media1.giphy.com/media/a0yDA3nCY0jV2vh3EJ/giphy.gif", df['link'])

In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from skimage import io

import pandas as pd
import re
from PIL import Image
from IPython.core.display import Image

#Names = sorted(set(df["Matchup"]))



def Allplot(name):
    Names = sorted(set(df["Matchup"]))


    play = df[(df["Matchup"]==name)]

    default = name

    plot_names = []
    use_names = []
    buttons=[]
    links = {}

    fig = make_subplots(
    rows=5, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    x_title= "Frame",
    specs=[[{"type": "table"}],
            [{"type": "table"}],
            [{"type": "scatter"}],
            [{"type": "scatter"}],
            [{"type": "scatter"}]],
    subplot_titles=("Press Score","Data","Velocity", "Acceleration","Separation"))



    fig.add_trace(
        go.Scatter(
            x=play["Frames"],
            y=play["Accel"],
            mode="lines",
            name="Acceleration"),
        row=4, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=play["Frames"],
            y=play["AccelMax"],
            mode="lines",
            name="Max Acceleration", line_width=3, opacity=1, line_dash="dot",line=dict(color='red', width=3)),
        row=4, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=[play['MaxAccelFrameAvg'].iloc[0], play['MaxAccelFrameAvg'].iloc[0]],
            y=[0,play['Accel'].max()+4],
            mode="lines",
            text='Avg. Max Acceleration Frame',
            name="Avg. Max Acceleration Frame",visible=(name==default), line_width=3, opacity=1, line_dash="dot",line=dict(color='blue', width=4)
        ),
        row=4, col=1
    )



    fig.add_trace(
        go.Scatter(
            x=play["Frames"],
            y=play["Velocity"],
            mode="lines",
            name="Velocity"),
        row=3, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=play["Frames"],
            y=play["dxMax"],
            mode="lines",
            name="Max Velocity", line_width=3, opacity=1, line_dash="dot",line=dict(color='red', width=3)),
        row=3, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=[play['MaxdxFrameAvg'].iloc[0], play['MaxdxFrameAvg'].iloc[0]],
            y=[play['Velocity'].min(),play['Velocity'].max()+4],
            mode="lines",
            text='Avg. Max Velocity Frame',
            name="Avg. Max Velocity Frame",visible=(name==default), line_width=3, opacity=1, line_dash="dot",line=dict(color='blue', width=4)
        ),
        row=3, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=play["Frames"],
            y=play["Separation"],
            mode="lines",
            name="Separation"),
        row=5, col=1
    )


    fig.add_trace(
        go.Scatter(
            x=[play['MaxSeparationFrameAvg'].iloc[0], play['MaxSeparationFrameAvg'].iloc[0]],
            y=[play['Separation'].min(),play['Separation'].max()+4],
            mode="lines",
            text='Avg. Max Separation Frame',
            name="Avg. Max Separation Frame",visible=(name==default), line_width=3, opacity=1, line_dash="dot",line=dict(color='blue', width=4)
        ),
        row=5, col=1
    )



    fig.add_trace(
        go.Table(
            header=dict(
                values=["playId", "Name", "route",
                        "Frames", "Accel", "Velocity",
                        "Separation"],
                font=dict(size=10),
                align="left"
            ),
            cells=dict(
                values=[play[k].tolist() for k in ["playId", "Name", "route",
                        "Frames", "Accel", "Velocity",
                        "Separation"]],
                align = "left")
        ),
        row=2, col=1
    )

    fig.add_trace(
        go.Table(
            header=dict(
                values=["Matchup","Press Score"],
                font=dict(size=10),
                align="left"
            ),
            cells=dict(
                values=[play[k].iloc[0] for k in ["Matchup","Press Score"]],
                line_color='darkslategray',
                fill=dict(color=['white', 'paleturquoise']),
                align=['left', 'center'],
                font_size=18,
                height=55)
        ),
        row=1, col=1
    )

    link = "'<img src =" + play['link'].iloc[0] + ">'"
    links.setdefault(name,[]).append(link)

    fig.update_layout(
        height=1000,
        width=700,
        showlegend=False,
    #    title_text= display(HTML(link)),
        margin = {'t':75, 'l':75})
    
    plot_names.extend([name]*10)


    fig.update_layout(showlegend=False,title=display(HTML(links[name][0])))
    return fig.show()

--------------------------------------------------------------------------------------------------------------------------

# Let's Visualize This

Below you will see the same graphic we showed earlier between Mike Evans and James Bradberry, except now we display Mike Evan's Acceleration, Velocity and Separation in the first 1.5 seconds of the play along with James Bradberry's Press Score.

**The Red Horizontal lines** = The average max for each metric that Mike Evans usually generates on a slant route in press coverage

**The Blue Vertical lines** = The average frame it takes Mike Evans to reach their max for each metric

**Frames** = 1/10 of a second

In [ ]:
Allplot('Mike Evans vs. James Bradberry')

## How to Read This:

1. **Press Score** - James Bradberry received a press grade of 61 against Mike Evans on this play, which means he was successful in disrupting the route to force an incompletion.

2. **Data** - Here you can see the raw data for Mike Evans in the first 15 frames after the ball is snapped

3. **Velocity**
   - **Maximum Velocity** - You can clearly see that James Bradberry slowed down Mike Evan's velocity to less than what he generally produces on slant routes during press coverage. 

   - **Maximum Velocity Frame** - Mike Evans maximum velocity frame happened sooner than expected, if the receiver were to beat the cornerback off the ball then we would expect the max velcoity frame at either 14 or 15 frames.

4. **Acceleration**
   - **Maximum Acceleration** - like velocity, you see that James Bradberry slowed down Mike Evan's acceleration to less than what he generally produces on slant routes during press coverage. 

   - **Maximum Acceleration Frame** - Mike Evans maximum acceleration frame happened sooner than expected, we would expect max acceleration to also occur later on in the route
   
5. **Separation**
   - **Maximum Separation Frame** - You can see his max separation frame occurs sooner than expected, meaning after initial contact, he was unable to fend off Bradberry.

# Beat the Press

Here's another example of Mike Evans in press coverage except this time, he beats it.

In [ ]:
Allplot('Mike Evans vs. Marshon Lattimore')

You can see Mike Evans won at the line of scrimmage. Compared to what he normally produces on go routes during press coverage, here's how he did.

- Max Velocity - Exceeded expected ✔️
- Max Velocity Frame - Exceeded expected ✔️
- Max Acceleration - Exceeded expected ✔️
- Max Acceleration Frame - Exceeded expected ✔️
- Max Separation Frame - Exceeded expected ✔️

Thus giving Marshon Lattimore a poor press grade of 22.4

In [ ]:
CBs = Finaldf1.filter(['displayName','nflId','position','Targeted','playId','epa'], axis=1).query('position == "CB" & Targeted == 1')
CBs = CBs.groupby(['displayName','nflId','position','playId'])['epa'].mean().reset_index()
CBs = CBs.groupby(['displayName','nflId','position'])['epa'].mean().reset_index()
CBs = CBs.set_index('nflId')
CBs = CBs.loc[~CBs.index.duplicated(keep='first')]

------------------------------------------------------

# Is This Noise?

Let's see who performed the best during press coverage, and also look at how our press score correlates with Defender Success and EPA

# Targeted Press Score Vs. Targeted Press Defender Success

In [ ]:
#var1 = 'Press_Success'
var1 = 'PressScore'
var2 = 'DefenderSuccess'

MAAE1 = PressScore1.groupby(['closestOpp_Id_']).agg({var1:[('mean', 'mean'),('count','count')],
                                                         var2:[('mean', 'mean')],}).reset_index(drop=False)
MAAE1.columns = MAAE1.columns.map('_'.join)

MAAE1.columns = MAAE1.columns.str.replace(var2 +'_mean' , var2)
MAAE1.columns = MAAE1.columns.str.replace(var1 + '_mean' , var1)
MAAE1.columns = MAAE1.columns.str.replace(var1 + '_count' , 'count')
MAAE1.columns = MAAE1.columns.str.replace('closestOpp_Id__' , 'closestOpp_Id_')

#MAAE1[['mean']] = MinMaxScaler().fit_transform(MAAE1[['mean']])
MAAE1[[var1]] = MinMaxScaler().fit_transform(MAAE1[[var1]])*100
MAAE1['displayName'] = MAAE1.closestOpp_Id_.map(CBs['displayName'])
#MAAE1['epa'] = MAAE1.closestOpp_Id_.map(CBs['epa'])
MAAE1 = MAAE1.sort_values(by=[var1], ascending=False).query('count > 12')
MAAE1.drop(['count'], axis=1, inplace=True)


print("Total players with more than 12 press coverage targeted snaps: ", len(MAAE1))

print("Top 10 Press Score")
display(HTML(MAAE1.head(10).to_html()))

fig, ax = plt.subplots(figsize=(15,10))
slope, intercept, r_value, pv, se = stats.linregress(MAAE1[var1], MAAE1[var2])

print(var1 + " vs. " + var2, " Correlation:", r_value)

sns.regplot(MAAE1[var1], MAAE1[var2], line_kws={'label':'$y=%3.7s*x+%3.7s$'%(slope, intercept)})
plt.xticks(fontsize=15)
plt.xlabel("Average " + var1)
plt.legend()
plt.yticks(fontsize=15)
plt.ylabel("Average " + var2)

We can see a decent positive correlation of .28 with Defender Success, which indicates that cornerbacks with high average press scores may force incompletions / interceptions more often.  

# Targeted Press Score Vs. Targeted Press EPA

In [ ]:
#var1 = 'Press_Success'
var1 = 'PressScore'
var2 = 'epa_mean'

MAAE1 = PressScore1.groupby(['closestOpp_Id_']).agg({var1:[('mean', 'mean'),('count','count')],
                                                         var2:[('mean', 'mean')],}).reset_index(drop=False)
MAAE1.columns = MAAE1.columns.map('_'.join)

MAAE1.columns = MAAE1.columns.str.replace(var2 +'_mean' , var2)
MAAE1.columns = MAAE1.columns.str.replace(var1 + '_mean' , var1)
MAAE1.columns = MAAE1.columns.str.replace(var1 + '_count' , 'count')
MAAE1.columns = MAAE1.columns.str.replace('closestOpp_Id__' , 'closestOpp_Id_')

#MAAE1[['mean']] = MinMaxScaler().fit_transform(MAAE1[['mean']])
MAAE1[[var1]] = MinMaxScaler().fit_transform(MAAE1[[var1]])*100
MAAE1['displayName'] = MAAE1.closestOpp_Id_.map(CBs['displayName'])
#MAAE1['epa'] = MAAE1.closestOpp_Id_.map(CBs['epa'])
MAAE1 = MAAE1.sort_values(by=[var1], ascending=False).query('count > 12')
MAAE1.drop(['count'], axis=1, inplace=True)


print(len(MAAE1))
display(HTML(MAAE1.head(10).to_html()))

fig, ax = plt.subplots(figsize=(15,10))
slope, intercept, r_value, pv, se = stats.linregress(MAAE1[var1], MAAE1[var2])

print(var1 + " vs. " + var2, " Correlation:", r_value)

sns.regplot(MAAE1[var1], MAAE1[var2], line_kws={'label':'$y=%3.7s*x+%3.7s$'%(slope, intercept)})
plt.xticks(fontsize=15)
plt.xlabel("Average " + var1)
plt.legend()
plt.yticks(fontsize=15)
plt.ylabel("Average " + var2)

We can see that a correlation of -.22 between Press Score and EPA given up on press coverage. This signals that there may be some relationship between a good press at the line of scrimmage and EPA.

---------------------------------------------------------------------------------------------------

In [ ]:
PressScore1 = PressScore.query('MPH_max > 5 & Total >= 10 & Total <= 15 & FramePct == 1 & Opp_Dist_COpp_start <= 3 & MovingForward > -1 & dx_COpp_first < .2 & Opp_Dist_COpp_last < 6 & route_ != "SCREEN"').reset_index(drop=True)





Cols = ['Opp_Dist_COpp_start', 'Opp_Dist_COpp_last', 'Opp_Dist_COpp_mean',  'Opp_Dist_COpp_max', 'MPH_max', 'MPH_var', 'MPH_mean', 'a_max', 'a_var', 'dx_mean', 'dx_var', 'dx_max','dy_mean', 'dy_var', 'dy_max', 'dx_COpp_first', 'dx_COpp_mean', 'dx_COpp_max', 'MaxAccelFrame_max', 'MaxdxFrame_max', 'MaxSeparationFrame_max','dx_dff','dy_dff','a_dff','MaxAccelbyFrame','MaxdxbyFrame','MaxSeparationbyFrame','MaxAccelDiffFrame_max','MaxSpeedDiffFrame_max','MaxdxDiffFrame_max','MaxdyDiffFrame_max']

for col in Cols:
    PressScore1[col + "_AE"] = PressScore1[col] - PressScore1.groupby(['nflId_','route_'])[col].transform('mean')

Cols = ['dx_max_AE','a_max_AE','MaxAccelFrame_max_AE',  'MaxdxFrame_max_AE','MaxSeparationFrame_max_AE']


X = PressScore1[Cols]
#X = pd.get_dummies(X)


#Cols = ['MaxdxFrame_max', 'MaxSeparationFrame_max', 'MaxAccelFrame_max_AE',  'MaxdxFrame_max_AE']

y = PressScore1['DefenderSuccess']


X[Cols] = SimpleImputer().fit_transform(X[Cols])
X[Cols] = StandardScaler().fit_transform(X[Cols])



PressScore1['PressScore'] = grid_clf_acc.predict_proba(X)[:,1]
PressScore1['Press_Success'] = grid_clf_acc.predict(X)
PressScore1[['PressScore']] = MinMaxScaler().fit_transform(PressScore1[['PressScore']])
PressScore1['PressScore'] = PressScore1['PressScore']*100

Press = PressScore1.filter(['gameId_','playId_','nflId_','dx_max_AE','dx_var_AE','dy_var_AE', 'a_max_AE','Opp_Dist_COpp_last_AE', 'MaxAccelFrame_max_AE',  'MaxdxFrame_max_AE','MaxSeparationFrame_max_AE','PressScore','Press_Success'], axis=1)
Press['Code'] = Press['gameId_'].astype(str) + "-" + Press['playId_'].astype(str) + "-" + Press['nflId_'].astype(str)
Press = Press.set_index('Code')
Press = Press.loc[~Press.index.duplicated(keep='first')]

OffBall['InitialDefDis'] = OffBall.groupby(['playId','nflId','closestOpp_Id'])['Opp_Dist_COpp'].transform('first')
OffBall['Press'] = np.where(OffBall['InitialDefDis'] < 3, 1,0)
OffBall.query('displayName == "Mike Evans"')


#df = OffBall.query('displayName == "Mike Evans" & gameId == 2018110402 & playId ==558	').head(10).filter(['playId','displayName','route','EventOrder','a'],axis=1).rename(columns={'EventOrder':'Frames'})
#df = OffBall.query('displayName == "Michael Thomas" & gameId == 2018102101 & playId == 2235	').head(10).filter(['playId','displayName','route','EventOrder','a'],axis=1).rename(columns={'EventOrder':'Frames'})
df = OffBall.query('displayName == "Michael Thomas" & gameId == 2018110409 & playId == 3891	| displayName == "Mike Evans" & gameId == 2018110402 & playId == 558 | displayName == "Tyreek Hill" & gameId == 2018100705 & playId == 395 | displayName == "Pierre Garcon" & gameId == 2018101500 & playId == 1249| displayName == "Michael Crabtree" & gameId == 2018102101 & playId == 3444 | displayName == "Devin Funchess" & gameId == 2018121700 & playId == 2048 | displayName == "Davante Adams" & gameId == 2018102810 & playId == 412| displayName == "Mike Evans" & gameId == 2018090906 & playId == 3051 |  displayName == "Tyreek Hill" & gameId == 2018091605 & playId == 2430 ').filter(['gameId','playId','nflId','displayName','route','EventOrder','a','dx','Opp_Dist_COpp'],axis=1)#.rename(columns={'EventOrder':'Frames'})

#df = OffBall.query('displayName == "Mike Evans" & gameId == 2018110402 & playId == 558').filter(['playId','displayName','route','EventOrder','a','dx'],axis=1)#.rename(columns={'EventOrder':'Frames'})

df.columns = ['gameId','playId','nflId','Name','route','Frames','Accel','Velocity','Separation']

df["Accel"] = df['Accel'].map(lambda x: np.round(x,3))
df["Velocity"] = df['Velocity'].map(lambda x: np.round(x,3))
df["Separation"] = df['Separation'].map(lambda x: np.round(x,3))

df['AccelMax'] = OffBall.groupby(['displayName','route','Press'])['MaxAccel'].transform('mean')
df["AccelMax"] = df['AccelMax'].map(lambda x: np.round(x,3))

df['dxMax'] = OffBall.groupby(['displayName','route','Press'])['Maxdx'].transform('mean')
df["dxMax"] = df['dxMax'].map(lambda x: np.round(x,3))

df['MaxSeparation'] = OffBall.groupby(['displayName','route','Press'])['MaxSeparation'].transform('mean')
df["MaxSeparation"] = df['MaxSeparation'].map(lambda x: np.round(x,3))

df['MaxSeparationFrameAvg'] = OffBall.groupby(['displayName','route','Press'])['MaxSeparationFrame'].transform('mean')
df["MaxSeparationFrameAvg"] = df['MaxSeparationFrameAvg'].map(lambda x: np.round(x,3))

df['MaxdxFrameAvg'] = OffBall.groupby(['displayName','route','Press'])['MaxdxFrame'].transform('mean')
df["MaxdxFrameAvg"] = df['MaxdxFrameAvg'].map(lambda x: np.round(x,3))


df['MaxAccelFrameAvg'] = OffBall.groupby(['displayName','route','Press'])['MaxAccelFrame'].transform('mean')
df["MaxAccelFrameAvg"] = df['MaxAccelFrameAvg'].map(lambda x: np.round(x,3))

df['Code'] = df['gameId'].astype(str) + "-" + df['playId'].astype(str) + "-" + df['nflId'].astype(str)

df["Press Score"] = df.Code.map(np.round(Press["PressScore"],3))



df['Matchup'] = np.where((df['Name'] == "Michael Thomas") & (df['gameId'] == 2018110409) & (df['playId'] == 3891), "Michael Thomas vs. Marcus Peters", 0)
df['Matchup'] = np.where((df['Name'] == "Pierre Garcon") & (df['gameId'] == 2018101500) & (df['playId'] == 1249), "Pierre Garcon vs. Kevin King", df['Matchup'])
df['Matchup'] = np.where((df['Name'] == "Michael Crabtree") & (df['gameId'] == 2018102101) & (df['playId'] == 3444), "Michael Crabtree vs. Ken Crawley", df['Matchup'])
df['Matchup'] = np.where((df['Name'] == "Devin Funchess") & (df['gameId'] == 2018121700) & (df['playId'] == 2048), "Devin Funchess vs. Eli Apple", df['Matchup'])
df['Matchup'] = np.where((df['Name'] == "Davante Adams") & (df['gameId'] == 2018102810) & (df['playId'] == 412), "Davante Adams vs. Marcus Peters", df['Matchup'])
df['Matchup'] = np.where((df['Name'] == "Tyreek Hill") & (df['gameId'] == 2018091605) & (df['playId'] == 2430), "Tyreek Hill vs. Artie Burns", df['Matchup'])


df['link'] = np.where(df["Matchup"] == "Michael Thomas vs. Marcus Peters", "https://media4.giphy.com/media/cXgBT9FPpHtGSxC4gY/giphy.gif", 0)
df['link'] = np.where(df["Matchup"] == "Pierre Garcon vs. Kevin King", "https://media3.giphy.com/media/Xdrtq0QgTiVGRBV1LH/giphy.gif", df['link'])
df['link'] = np.where(df["Matchup"] == "Michael Crabtree vs. Ken Crawley", "https://media0.giphy.com/media/CqO673JlHLKdnnEoRp/giphy.gif", df['link'])
df['link'] = np.where(df["Matchup"] == "Devin Funchess vs. Eli Apple", "https://media1.giphy.com/media/n8KgzfQQCHhFxbJUO5/giphy.gif", df['link'])

df['link'] = np.where(df["Matchup"] == "Davante Adams vs. Marcus Peters", "https://media2.giphy.com/media/eiWqmciwHLKiBdLmXT/giphy.gif", df['link'])
df['link'] = np.where(df["Matchup"] == "Tyreek Hill vs. Artie Burns", "https://media1.giphy.com/media/a0yDA3nCY0jV2vh3EJ/giphy.gif", df['link'])

# Non-Targeted Press Score

We've looked at targeted press score, but would it be possible measure press for receivers that weren't targeted?

We only take into account the initial 1.5 seconds, so this method can be applied to non-targeted receivers.

Let's evaluate the highest graded non-targeted press play.

In [ ]:
Allplot('Pierre Garcon vs. Kevin King')

Kevin King pressed Pierre Garcon so well that his velocity actually went into the negative at one point.

# Final Thoughts:

It looks like we may have found a way to quanitfy a cornerbacks ability to press a receiver at the line of scrimmage. Note that a high press score may not be indicative of the cornerbacks overall ability, however this is valuable information for GMs and defensive coordinators that are looking to find undervalued free agents / trade options that better fit their press heavy defensive scheme.



## Other Work:

1. [Shadow Cornerback + Coverage Analysis](https://www.kaggle.com/jdruzzi/shadow-cornerback-coverage-analysis)

2. [Defender Bite Velocity on Play-Action](https://www.kaggle.com/jdruzzi/defender-bite-velocity-on-play-action)

3. [Pass Coverage Classification](https://www.kaggle.com/jdruzzi/pass-coverage-classification-80-recall)

4. [Quantifying Press Coverage](https://www.kaggle.com/jdruzzi/quantifying-press-coverage-ability)

5. [Defender Tendencies: One-Cut Routes + Double Moves](https://www.kaggle.com/jdruzzi/defender-tendencies-one-cut-routes-double-moves)

## Data:

[Revised BDB Data](https://www.kaggle.com/jdruzzi/revised-bdb-data)

# Bonus Examples

In [ ]:
Allplot("Michael Thomas vs. Marcus Peters")

In [ ]:
Allplot("Devin Funchess vs. Eli Apple")

In [ ]:
Allplot("Tyreek Hill vs. Artie Burns")

In [ ]:
Allplot("Davante Adams vs. Marcus Peters")